In [1]:
import pandas as pd
import os

In [2]:
df_path = "data/NU_340297544_01AGO2023_31AGO2023.csv"
df = pd.read_csv(df_path, sep=",", encoding="latin-1")
df = df.rename({'DescriÃÂ§ÃÂ£o': 'Descricao'}, axis = 1)
df.head()

,Data,Valor,Identificador,Descricao
0,03/08/2023,-4.0,64cb88e6-f81f-4b43-b9f8-dc1dc8f7d379,Compra no dÃÂ©bito - Suco Tal
1,03/08/2023,-9.9,64cbc2f3-9d0d-4795-8af0-b6910dc280ed,Compra no dÃÂ©bito - Mc Donalds
2,03/08/2023,-3.5,64cc3384-dde7-492f-b0e7-dab258bd4417,Compra no dÃÂ©bito - Dellicata Paes e Tenta
3,04/08/2023,800.0,64cd05e2-bd32-4023-bc2e-b94b77507cc6,TransferÃÂªncia recebida pelo Pix - WYNK SOLU...
4,04/08/2023,-2.5,64cd1991-67bf-4908-9804-ba7d5fa79446,TransferÃÂªncia enviada pelo Pix - Samuel Mot...


In [3]:
def get_api_key():
    with open('API_KEY.txt', 'r') as file:
        api_key = file.read().strip()
    return api_key

In [16]:
import openai
import os
import time

def get_ai_response(description):
    openai.api_key = get_api_key()


    # Create a prompt
    prompt = "I'll give you a description in portuguese-br, and you will give me the corresponding categories. Possible Categories: ( 'Alimentacao', 'Pix', 'Outros', 'Compras online', 'Pagar Fatura'). Description:\n" + "\n".join(description)

    # Make an API call for the current batch
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150,
        temperature=0
    )

    response = response.choices[0].text.strip()
    categorie = response.split("\n")[-1].strip() 

    # Pause for a minute to respect the rate limit
    time.sleep(25)

    return categorie

In [17]:
categorias = []
descricao = []
for linha in df['Descricao']:
    print(linha)
    categoria = get_ai_response(linha)
    print(categoria)
    categorias.append(categoria)
print(categorias)
df['Categoria'] = categorias

Compra no dÃÂ©bito - Suco Tal


In [6]:
df

,Data,Valor,Identificador,Descricao
0,03/08/2023,-4.00,64cb88e6-f81f-4b43-b9f8-dc1dc8f7d379,Compra no dÃÂ©bito - Suco Tal
1,03/08/2023,-9.90,64cbc2f3-9d0d-4795-8af0-b6910dc280ed,Compra no dÃÂ©bito - Mc Donalds
2,03/08/2023,-3.50,64cc3384-dde7-492f-b0e7-dab258bd4417,Compra no dÃÂ©bito - Dellicata Paes e Tenta
3,04/08/2023,800.00,64cd05e2-bd32-4023-bc2e-b94b77507cc6,TransferÃÂªncia recebida pelo Pix - WYNK SOLU...
4,04/08/2023,-2.50,64cd1991-67bf-4908-9804-ba7d5fa79446,TransferÃÂªncia enviada pelo Pix - Samuel Mot...
5,04/08/2023,-610.72,64cd21c5-ccf2-4b1b-85b8-388f8b2a5f60,Pagamento de fatura
6,04/08/2023,-4.42,64cd84f6-a720-4a63-aa19-2470fa1a697e,Compra no dÃÂ©bito - Dellicata Paes e Tenta
7,05/08/2023,-25.00,64ce5e0f-badf-4f48-8c19-ebc9cbdbeba4,TransferÃÂªncia enviada pelo Pix - Nilza Paes...
8,05/08/2023,-2.16,64cec029-d871-4103-977c-fc28b0f3c590,TransferÃÂªncia enviada pelo Pix - Samuel Mot...
9,05/08/2023,-50.00,64cec06d-e26d-4ff4-b8fa-3633b13d28b2,AplicaÃÂ§ÃÂ£o RDB
